# Submission Details

* __Student__: Shlomi Ben-Shushan

* __Course__: Deep Learning (22961)

* __Assignment__: Maman 11

### Video Link

TODO: add link

### Imports

In [ ]:
from typing import Tuple
from torch import Tensor, tensor, zeros

### Question A

In [ ]:
def align_dimensions(A: Tensor, B: Tensor) -> Tensor:
    # Get tensor shapes as a lists
    A_shape = list(A.shape)
    B_shape = list(B.shape)

    # If A has more dimensions than B, raise an error because it's not possible to expand.
    if len(A_shape) > len(B_shape):
        raise ValueError(f"Cannot expand shape {A.shape} to {B.shape}")

    # Add leading 1s to A's shape to match the number of dimensions in B.
    while len(A_shape) < len(B_shape):
        A_shape.insert(0, 1)

    # Reshape A to match the new shape with leading 1s.
    return A.view(*A_shape)

def expand_axis(A: Tensor, dim: int, times: int) -> Tensor:
    if times <= 1:
        return A  # No need to expand if the times value is 1 or less.

    # Create a new shape, where only the specified dimension is expanded.
    return A.expand(*[-1 if i != dim else times for i in range(A.dim())])

def expand_as(A: Tensor, B: Tensor) -> Tensor:
    # Align A's dimensions to match B's by adding leading singleton dimensions.
    A = align_dimensions(A, B)

    # Clone A to avoid modifying the original tensor.
    out = A.clone()

    # For each dimension of B, expand A if necessary.
    B_size = B.size()
    for i in range(len(B_size)):
        if out.size(i) == 1 and B_size[i] != 1:
            out = expand_axis(out, i, B_size[i])

    # Check if the resulting shape matches B's shape; raise an error if not.
    if out.shape != B.shape:
        raise ValueError(f"Cannot expand tensor of shape {A.shape} to {B.shape}")

    # Return the expanded tensor.
    return out

In [105]:
A = tensor([[1], [2], [3]])  # Shape (3,1)
B = zeros((3, 4))            # Shape (3,4)
A_BACKUP = A.clone()
B_BACKUP = B.clone()
expand_as(A, B)
assert B.equal(B_BACKUP) and A.equal(A_BACKUP), "Test case 1 failed"

A = tensor([[1], [2], [3]])  # Shape (3,1)
B = zeros((3, 4))            # Shape (3,4)
expanded_A = expand_as(A, B)
assert expanded_A.shape == B.shape, "Test case 2 failed"

A = tensor([1, 2, 3])        # Shape (3,)
B = zeros((3, 4))            # Shape (3,4)
try:
    expanded_A = expand_as(A, B)
except ValueError:
    pass
else:
    assert False, "Test case 3 failed"

A = tensor([1])              # Shape (1,)
B = zeros((2, 3, 4))         # Shape (2,3,4)
expanded_A = expand_as(A, B)
assert expanded_A.shape == B.shape, "Test case 4 failed"

try:
    A = tensor([1, 2])       # Shape (2,)
    B = zeros((3, 4))        # Shape (3,4)
    expand_as(A, B)
    assert False, "Test case 5 failed: Expected ValueError"
except ValueError:
    pass

print("All tests passed!")

All tests passed!


### Question B

In [ ]:
def is_tensor_empty(t: Tensor) -> bool:
    return t is None or t.numel() == 0

def are_broadcastable(A: Tensor, B: Tensor) -> Tuple[bool, Tuple[int]]:
    # Ensure no empty tensor was provided
    if is_tensor_empty(A) or is_tensor_empty(B):
        return False, None

    # Check tensor sizes and ensure A_size is shorter
    A_size, B_size = sorted([A.size(), B.size()], key=len)

    # Start with the larger tensor's shape.
    dim = list(B_size)

    # Iterate over dimensions from the last (rightmost) to the first (leftmost).
    for i in range(1, len(A_size) + 1):

        # If dimensions match or A_size has 1, use B_size's dimension.
        if A_size[-i] == B_size[-i] or A_size[-i] == 1:
            dim[-i] = B_size[-i]

        # If B_size has 1, use A_size's dimension.
        elif B_size[-i] == 1:
            dim[-i] = A_size[-i]

        # If dimensions are incompatible, broadcasting is not possible.
        else:
            return False, None

    # Return success and the resulting broadcast shape.
    return True, tuple(dim)


In [107]:
assert are_broadcastable(tensor([1]), zeros((2, 3, 4))) == (True, (2, 3, 4)), "Test case 1 failed"
assert are_broadcastable(tensor([[1], [2], [3]]), zeros((3, 4))) == (True, (3, 4)), "Test case 2 failed"
assert are_broadcastable(tensor([1, 2]), zeros((3, 4))) == (False, None), "Test case 3 failed"
assert are_broadcastable(tensor([[1, 2, 3]]), zeros((3,))) == (True, (1, 3)), "Test case 4 failed"
print("All tests passed!")

All tests passed!


### Question C

In [ ]:
def broadcast_tensors(A: Tensor, B: Tensor) -> Tuple[Tensor]:
    broadcastable, result_shape = are_broadcastable(A, B)
    if not broadcastable:
        raise ValueError("Tensors cannot be broadcasted together")

    # Assume the given tensors are broadcastable and use expand_as expand each given tensor
    expanded_A = expand_as(A, zeros(result_shape))
    expanded_B = expand_as(B, zeros(result_shape))
    return expanded_A, expanded_B

In [ ]:
A = tensor([1])              # Shape (1,)
B = zeros((2, 3, 4))         # Shape (2,3,4)
expanded_A, expanded_B = broadcast_tensors(A, B)
assert expanded_A.shape == expanded_B.shape == (2, 3, 4), "Test case 1 failed"

A = tensor([[1], [2], [3]])  # Shape (3,1)
B = zeros((3, 4))            # Shape (3,4)
expanded_A, expanded_B = broadcast_tensors(A, B)
assert expanded_A.shape == expanded_B.shape == (3, 4), "Test case 2 failed"

try:
    A = tensor([1, 2])       # Shape (2,)
    B = zeros((3, 4))        # Shape (3,4)
    broadcast_tensors(A, B)  # Should raise error
    assert False, "Test case 3 failed: Expected ValueError"
except ValueError:
    pass

print("All tests passed!")

All tests passed!


### Question D

In [103]:
from torch import allclose, broadcast_tensors as torch_broadcast_tensors

test_cases = [
    (tensor([1]), zeros((2, 3, 4)), "Broadcasting scalar to (2,3,4)"),
    (tensor([[1], [2], [3]]), zeros((3, 4)), "Column vector to (3,4)"),
    (tensor([1, 2, 3]), zeros((3, 1)), "Row vector to (3,1)"),
    (tensor([[1, 2, 3]]), zeros((1, 3)), "1-row matrix to (1,3)"),
    (tensor([1, 1, 1]), zeros((3, 3)), "1D vector broadcast to (3,3)"),
    (tensor([[1], [2]]), zeros((2, 3)), "Column vector to (2,3)"),
    (tensor([[1, 2, 3]]), zeros((2, 3)), "Row vector to (2,3)"),
    (tensor([[[1]], [[2]]]), zeros((2, 3, 4)), "(2,1,1) to (2,3,4)"),
    (tensor([1, 2, 3, 4]), zeros((1, 4)), "Vector to (1,4)"),
    (tensor([[1, 2], [3, 4]]), zeros((2, 2)), "No broadcasting needed"),
    (tensor([[1], [2], [3]]), zeros((1, 3, 4)), "Expanding from (3,1) to (1,3,4)"),
    (tensor([1, 2, 3]).view(3, 1, 1), zeros((3, 1, 2)), "Broadcasting (3,) to (3,1,2)"),
    (tensor([[1, 2, 3], [4, 5, 6]]).view(2, 3, 1), zeros((2, 3, 4)), "Expanding to third dim"),
    (tensor([1, 2, 3, 4]), zeros((3, 1, 4)), "Matching last dimension"),
    (tensor([[[1, 2, 3]]]), zeros((2, 3, 3)), "Expanding singleton dimension"),
]

try:
    for A, B, description in test_cases:
        expanded_A, expanded_B = broadcast_tensors(A, B)
        expected_A, expected_B = torch_broadcast_tensors(A, B)
        assert allclose(expanded_A, expected_A), "Mismatch in broadcasted A"
        assert allclose(expanded_B, expected_B), "Mismatch in broadcasted B"
except:
    print(f"Test '{description}' failed!")
else:
    print("All tests passed!")

All tests passed!
